*Copyright 2025 Jaeyoung Chun / Winning Twelve*

You may not make copies of this and use or distribute it for any purpose.

# Vector Store

## Loading

In [1]:
from langchain_community.document_loaders import JSONLoader

In [2]:
# define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["source"] = record["metadata"]["source"]
    metadata["title"] = record["metadata"]["title"]
    return metadata

loader = JSONLoader(
    file_path="./data/news.json",
    jq_schema=".[]",
    content_key="page_content",
    metadata_func=metadata_func
)

In [3]:
data = loader.load()

In [4]:
len(data)

80

In [5]:
data[0]

Document(metadata={'source': 'https://www.newsis.com/view/NISX20250202_0003049875', 'seq_num': 1, 'title': '광주 미드필더 정호연, MLS 미네소타 이적…정상빈과 한솥밥'}, page_content='광주 미드필더 정호연, MLS 미네소타 이적…정상빈과 한솥밥광주 미드필더 정호연, MLS 미네소타 이적…정상빈과 한솥밥등록 2025.02.02 12:59:06[서울=뉴시스] 미국 프로축구 메이저리그사커(MLS) 미네소타 유나이티드의 정호연. (사진=미네소타 유나이티드 SNS 캡처) *재판매 및 DB 금지[서울=뉴시스] 김진엽 기자 = 프로축구 K리그1 광주FC 미드필더 정호연(25)이 미국 프로축구 메이저리그사커(MLS) 미네소타 유나이티드로 이적했다.미네소타는 2일(한국 시간) 구단 공식 채널을 통해 정호연 영입 소식을 전했다.구단에 따르면 계약 기간은 2027년 12월까지며, 1년 더 연장하는 옵션이 포함됐다.이로써 미네소타는 한국인 선수 2명을 보유하게 됐다.지난 2023년부터 뛰고 있는 윙 정상빈에 이어 미드필더 정호연까지 가세했다.정호연은 지난 2022시즌 광주 유니폼을 입고 프로 무대를 밟았다.당시 K리그2 소속이었던 광주의 주전으로 활약했고, K리그1으로 승격한 2023시즌에는 34경기 2골4도움으로 영플레이어상까지 받았다.2023년 열린 항저우 아시안게임에서 금메달을 획득해 병역 혜택까지 받은 그는 지난해 3월에는 처음 성인 국가대표로도 선발된 바 있다.정호연은 "미네소타에 합류해 정말 기쁘다. 새로운 도전이 기대되며 기회를 줘서 고맙다"며 "새로운 환경에서 팀에 기여하고발전하는 모습을 보이기 위해 최선을 다할 것"이라고 전했다.지난 2015년 창단한 미네소타는 지난 시즌 MLS 서부 콘퍼런스 14개 팀 중 6위를 기록했다.2025시즌 MLS는 이달 하순 개막한다.')

## Splitting

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3500,
    chunk_overlap=0
)

In [8]:
docs = text_splitter.split_documents(data)

In [9]:
len(docs)

80

## Embeddings

In [10]:
from langchain_openai import OpenAIEmbeddings

In [11]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

## Vector Store

In [12]:
# purge previous db
import shutil
shutil.rmtree("./chroma_db", ignore_errors=True)

In [13]:
from langchain_chroma import Chroma

In [14]:
%%time

db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

CPU times: user 624 ms, sys: 261 ms, total: 885 ms
Wall time: 2.94 s


## Similarity Search

In [15]:
def summarize_results(results):    
    for result in results:
        if type(result) == tuple:
            result, score = result
        else:
            score = None
        print("=" * 80)
        print("Title: ", result.metadata["title"])
        print("Source:", result.metadata["source"])
        if score:
            print("Score: ", score)

In [16]:
query = "2025년 1월에 있었던 챔피언스리그 벤피카 vs. 바르셀로나 경기에 대해 요약해줘."

### Basic Search

In [17]:
results = db.similarity_search(query)

In [18]:
results[0]

Document(id='57c397a5-7970-4fec-baf6-af9273246087', metadata={'seq_num': 48, 'source': 'https://www.newsis.com/view/NISX20250122_0003041476', 'title': '바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출'}, page_content='바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출등록 2025.01.22 10:33:04[리스본=AP/뉴시스] 스페인 프로축구 프리메라리가 바르셀로나의 하피냐. 2025. 1. 21.[서울=뉴시스] 하근수 기자 = 스페인 프로축구 프리메라리가 바르셀로나가 벤피카(포르투갈)와 도합 9골이 터진 난타전 끝에극적인 역전승을 거두면서 유럽축구연맹(UEFA) 챔피언스리그(UCL) 16강에 진출했다.바르셀로나는 22일(한국 시간) 포르투갈 리스본의 이스타디우 다 루스에서 열린 벤피카와의 대회 리그 페이즈 7차전 원정 경기에서 5-4로 이겼다.이날 승리로 바르셀로나는 승점 18(6승 1패)로 2위를 달렸고, 오는 31일 8차전 유벤투스(이탈리아)와의 맞대결 결과와 관계없이 16강행을 확정했다.UCL 리그 페이즈는 36개 팀 중 1~8위가 16강 직행, 9~24위가 16강 플레이오프를 치른다.바르셀로나는 리그 페이즈에서 단 1패밖에 내주지 않으며 순조롭게 16강 진출을 확정했다.안방에서 패배한 벤피카는 승점 10(3승 1무 3패)으로 18위에 머물렀다.벤피카는 전반전에 3-1, 후반전에 4-2로 앞서갔으나 끝내 역전을 허용하며 무릎을 꿇었다.[리스본=AP/뉴시스] 스페인 프로축구 프리메라리가 바르셀로나의 하피냐(가운데)가 득점 이후 동료들과 세리머니를 펼치고있다. 2025. 1. 21.벤피카가 킥오프 이후 곧장 균형을 깼다.전반 2분 반젤리스 파블리디스가 알바로 페르난데스가 건넨 크로스를 슈팅해 골망을 갈랐다.바르셀로나는 전반 13분 로베르트 레반도프스키의 페

In [19]:
summarize_results(results)

Title:  바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출
Source: https://www.newsis.com/view/NISX20250122_0003041476
Title:  음바페, 스페인 무대 첫 해트트릭…레알 마드리드 완승 이끌어
Source: https://www.newsis.com/view/NISX20250126_0003046718
Title:  레알서 첫 해트트릭 기록한 음바페, 라리가 이달의 선수
Source: https://www.newsis.com/view/NISX20250201_0003049492
Title:  즈베즈다 설영우, UCL서 시즌 5호 도움…황인범 부상 복귀
Source: https://www.newsis.com/view/NISX20250130_0003047794


### Search with Score

In [20]:
# similarity search with distance.
results = db.similarity_search_with_score(query)

In [21]:
summarize_results(results)

Title:  바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출
Source: https://www.newsis.com/view/NISX20250122_0003041476
Score:  0.8085647487907384
Title:  음바페, 스페인 무대 첫 해트트릭…레알 마드리드 완승 이끌어
Source: https://www.newsis.com/view/NISX20250126_0003046718
Score:  1.1104447647971056
Title:  레알서 첫 해트트릭 기록한 음바페, 라리가 이달의 선수
Source: https://www.newsis.com/view/NISX20250201_0003049492
Score:  1.195515106925064
Title:  즈베즈다 설영우, UCL서 시즌 5호 도움…황인범 부상 복귀
Source: https://www.newsis.com/view/NISX20250130_0003047794
Score:  1.21378609355432


### Search with Relevance Score

In [22]:
# return docs and relevance scores in the range [0, 1].
results = db.similarity_search_with_relevance_scores(query)

In [23]:
summarize_results(results)

Title:  바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출
Source: https://www.newsis.com/view/NISX20250122_0003041476
Score:  0.42825838310167164
Title:  음바페, 스페인 무대 첫 해트트릭…레알 마드리드 완승 이끌어
Source: https://www.newsis.com/view/NISX20250126_0003046718
Score:  0.21479697667886588
Title:  레알서 첫 해트트릭 기록한 음바페, 라리가 이달의 선수
Source: https://www.newsis.com/view/NISX20250201_0003049492
Score:  0.15464316088232688
Title:  즈베즈다 설영우, UCL서 시즌 5호 도움…황인범 부상 복귀
Source: https://www.newsis.com/view/NISX20250130_0003047794
Score:  0.1417236223378111


## Retriever

### Basic

In [24]:
retriever = db.as_retriever(
    search_type="similarity",
)

In [25]:
results = retriever.invoke(query)

In [26]:
summarize_results(results)

Title:  바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출
Source: https://www.newsis.com/view/NISX20250122_0003041476
Title:  음바페, 스페인 무대 첫 해트트릭…레알 마드리드 완승 이끌어
Source: https://www.newsis.com/view/NISX20250126_0003046718
Title:  레알서 첫 해트트릭 기록한 음바페, 라리가 이달의 선수
Source: https://www.newsis.com/view/NISX20250201_0003049492
Title:  즈베즈다 설영우, UCL서 시즌 5호 도움…황인범 부상 복귀
Source: https://www.newsis.com/view/NISX20250130_0003047794


### Most Relevant

In [27]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 1}
)

In [28]:
results = retriever.invoke(query)

In [29]:
summarize_results(results)

Title:  바르셀로나, 벤피카에 5-4 대역전승…UCL 16강 진출
Source: https://www.newsis.com/view/NISX20250122_0003041476
